In [1]:
import sys
import itertools
import pandas as pd
from collections import Counter

sys.path.append('../src/')
from utils.parsing import get_users
from utils.auth_data import token

In [4]:
items = pd.read_parquet('../data/processed/items.parquet.gzip')

In [5]:
flat_list = list(itertools.chain(*items.likers.to_list()))
user_ids = list(set(flat_list))

In [6]:
users_info = get_users(user_ids, 1000)
# counting amount of likes for each user
likes_number = Counter(flat_list)
users = pd.DataFrame(users_info)
users = users.sort_values('id').reset_index(drop=True)
users['likes'] = list(dict(sorted(likes_number.items())).values())
users

100%|██████████| 361/361 [04:57<00:00,  1.21it/s]


,id,bdate,photo_max,sex,first_name,last_name,can_access_closed,is_closed,city,country,deactivated,likes
0,356,18.6,https://sun9-17.userapi.com/s/v1/if1/B81YNFSHb...,1,Арина,Ускова,True,False,"{'id': 2, 'title': 'Санкт-Петербург'}","{'id': 1, 'title': 'Россия'}",NaN,1
1,8904,NaN,https://sun9-5.userapi.com/s/v1/if1/i8C_nD6RQc...,1,Ольга,Паня,True,False,"{'id': 2, 'title': 'Санкт-Петербург'}","{'id': 1, 'title': 'Россия'}",NaN,1
2,11946,NaN,https://sun9-41.userapi.com/s/v1/ig2/KGV4KtNi0...,1,Marina,Groshikova,True,False,"{'id': 2, 'title': 'Санкт-Петербург'}","{'id': 1, 'title': 'Россия'}",NaN,8
3,22909,NaN,https://sun9-65.userapi.com/s/v1/if1/Q521cNwwv...,2,Михаил,Шахнов,False,True,"{'id': 2, 'title': 'Санкт-Петербург'}","{'id': 1, 'title': 'Россия'}",NaN,1
4,38005,23.9,https://sun9-71.userapi.com/s/v1/ig2/pJUu-N8jw...,1,Наташа,Балашова,True,False,"{'id': 1, 'title': 'Москва'}","{'id': 1, 'title': 'Россия'}",NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...
360068,793463428,28.5.1990,https://sun9-75.userapi.com/McdpcTNn6Mp4mz_rP2...,2,Alkazar,Kobrinec,True,False,NaN,NaN,banned,1
360069,793464868,8.10.1980,https://sun9-75.userapi.com/McdpcTNn6Mp4mz_rP2...,2,Pyadysh,Volodeeva,True,False,NaN,NaN,banned,1
360070,793466061,29.3.1989,https://sun9-75.userapi.com/McdpcTNn6Mp4mz_rP2...,2,Akepsij,Miselev,True,False,NaN,NaN,banned,1
360071,793821044,22.4.2002,https://sun9-75.userapi.com/s/v1/ig2/g6bo7osCe...,1,愚か者鏡,愚か者鏡,False,True,NaN,NaN,NaN,1


In [7]:
# Adding age feature
ages = []
for row in users.bdate.str.split('.'):
    if type(row) == list:
        if len(row) == 3:
            ages.append(row[2])
        else:
            ages.append(-1)
    else:
        ages.append(-1)

users['age'] = ages
users.age = users.age.astype('int')
users.age = 2023 - users.age
users.age = users.age.replace(2024,-1)  # Replace any invalid ages 2024 with -1
users = users.drop(columns=['can_access_closed', 'bdate'])

# Drop users with <= 5 likes (female), <= 2 likes (male), or unspecified gender
users = users.drop(index=users.query('sex == 1 and likes <= 5').index)
users = users.drop(index=users.query('sex == 2 and likes <= 2').index)
users = users.drop(index=users.query('sex == 0').index)

# Drop banned/deleted users and unneeded 'deactivated' column
users = users.query('deactivated != "banned" and deactivated != "deleted"').reset_index(drop=True)
users = users.drop(columns='deactivated')

# Drop users with default avatar
users = users.drop(index=users.query("photo_max == 'https://vk.com/images/camera_200.png'").index)
users = users.query('is_closed == False').reset_index(drop=True)

# Drop names
users = users.drop(columns=['first_name', 'last_name'])
users

,id,photo_max,sex,is_closed,city,country,likes,age
0,11946,https://sun9-41.userapi.com/s/v1/ig2/KGV4KtNi0...,1,False,"{'id': 2, 'title': 'Санкт-Петербург'}","{'id': 1, 'title': 'Россия'}",8,-1
1,62102,https://sun9-51.userapi.com/s/v1/ig2/iu0M-ERxn...,1,False,NaN,NaN,9,35
2,662418,https://sun9-64.userapi.com/s/v1/if2/gxSDA6_Q6...,2,False,"{'id': 1964269, 'title': 'Hong Kong'}","{'id': 97, 'title': 'Китай'}",16,-1
3,1277906,https://sun9-19.userapi.com/s/v1/if1/yzq3SNC64...,1,False,"{'id': 1, 'title': 'Москва'}","{'id': 1, 'title': 'Россия'}",10,-1
4,1345035,https://sun9-26.userapi.com/s/v1/if1/1xpNC-mZ-...,1,False,"{'id': 1, 'title': 'Москва'}","{'id': 1, 'title': 'Россия'}",17,-1
...,...,...,...,...,...,...,...,...
71048,785828149,https://sun9-3.userapi.com/s/v1/ig2/EViT7gFq4m...,1,False,NaN,NaN,145,18
71049,786259561,https://sun9-60.userapi.com/s/v1/ig2/0p6oXfDpH...,2,False,NaN,NaN,3,-1
71050,787242273,https://sun9-26.userapi.com/s/v1/ig2/RR7oi_6Ka...,1,False,"{'id': 1, 'title': 'Москва'}","{'id': 1, 'title': 'Россия'}",10,24
71051,788594701,https://sun9-78.userapi.com/s/v1/ig2/lS-z-7d6n...,1,False,NaN,NaN,14,23
